In [1]:
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans

# Kmeans Clustering on YearPredictionMSD

In [2]:
# reading data
data = pd.read_csv("year_prediction.csv")
data = data.rename(index=str, columns={"label":"year"})

In [3]:
# separate input attributes and output into different dataframes
X = data.iloc[:,1:]
Y = data.iloc[:,0]

# normalized the training set X
scaler = StandardScaler()
scaler.fit(X)
X_std = scaler.transform(X)

num_of_data, _ = X_std.shape

In [121]:
num_of_rep = 20
res = np.zeros(num_of_rep)
num_clusters = 10

for t in range(num_of_rep):
    kmeans = KMeans(n_clusters = num_clusters)
    kmeans.fit(X_std)
    record = kmeans.inertia_ / num_of_data
    res[t] = record
    print(record)

name_str = "./res/kmeans/kmeans-"+str(num_clusters)+"c.npy"    
np.save(name_str, res)

71.65047255247167
71.65037839463994
71.65038445285383
71.65043678725971
71.65055173588914
71.65043621527752
71.65043481087723
71.65066602130086
71.65744870527136
71.65045610855871
71.65057260055757
71.65039727026986
71.65057172896067
71.65042962393274
71.66007752663573
71.6505658605916
71.65057189113001
71.65051012927415
71.65057042708213
71.65041435398359


## Implement DistDimKmeans (Ding et al 2016)

In [5]:
def decode(number, b, t):
    res = []
    for _ in range(t):
        r = number % b
        res.append(r)
        number = (number - r) // b
    res.reverse()
    return res

def encode(number_list, b, t):
    res = 0
    number_list.reverse()
    for i in range(t):
        res += number_list[i] * (b ** i)
    return res

def distkmeans(D_list, w_list, n_c = 10):
    D = np.hstack(D_list)
    num_of_party = len(D_list)
    num_of_data, _ = D_list[0].shape
    kmeans_list = []
    total_dim = 0
    label_list = []
    for j in range(num_of_party):
        kmeans_list.append(KMeans(n_clusters = n_c))
        _, party_dim = D_list[j].shape
        total_dim += party_dim
        label = kmeans_list[j].fit_predict(D_list[j], sample_weight = w_list)
        label_list.append(label)
    
    grids_number = n_c ** num_of_party
    center_list = np.zeros((grids_number, total_dim))
    center_weights = np.zeros(grids_number)
    
    for h in range(grids_number):
        h_decode = decode(h, n_c, num_of_party)
        temp = []
        for j in range(num_of_party):
            temp.append((kmeans_list[j].cluster_centers_)[h_decode[j],:])
        center_list[h, :] = np.concatenate(temp)
    
    for i in range(num_of_data):
        temp = [l[i] for l in label_list]
        idx = encode(temp, n_c, num_of_party)
        center_weights[idx] += w_list[i]
    
    # normalize center_weights to 1
    center_weights = center_weights / np.sum(center_weights)
    
    server_kmeans = KMeans(n_clusters = n_c)
    server_kmeans.fit(center_list, sample_weight = center_weights)
    return server_kmeans

## Implement Coreset Construction

In [7]:
def uniform_kmeans(m, D_list, n_c = 10):
    D = np.hstack(D_list)
    D_df = pd.DataFrame(D)
    C = D_df.sample(n=m, replace=False)
    C = C.to_numpy()
    return C

def coreset_kmeans(m, D_list, n_c = 10):
    alpha = 2
    D = np.hstack(D_list)
    num_of_party = len(D_list)
    num_of_data, _ = D_list[0].shape
    kmeans_list = []
    label_list = []
    groupcost_list = []
    groupcount_list = []
    sensitivity = np.zeros((num_of_data, num_of_party))
    for j in range(num_of_party):
        kmeans_list.append(KMeans(n_clusters = n_c))
        label = kmeans_list[j].fit_predict(D_list[j])
        label_list.append(label)
        groupcost = np.zeros(n_c)
        groupcount = np.zeros(n_c)
        cost = kmeans_list[j].inertia_ / num_of_data
        t = kmeans_list[j].transform(D_list[j])
        for i in range(num_of_data):
            groupcount[label[i]] += 1
            groupcost[label[i]] += t[i,label[i]] ** 2
        for i in range(num_of_data):
            sensitivity[i,j] = alpha * (t[i,label[i]] ** 2) / cost \
                        + 2 * alpha * groupcost[label[i]] / (groupcount[label[i]] * cost) + 4 * num_of_data / groupcount[label[i]]
    s = np.sum(sensitivity, axis=1)
    D_df = pd.DataFrame(np.hstack((D, (1/s).reshape(-1,1))))
    C = D_df.sample(n=m, replace=False, weights=s)
    C = C.to_numpy()
    data = C[:,:-1]
    weights = C[:,-1]
    return data, weights

In [125]:
num_of_rep = 20
res = np.zeros(num_of_rep)
num_clusters = 10
size = 1000

X1 = X_std[:,:30]
X2 = X_std[:,30:60]
X3 = X_std[:,60:]

X_list = [X1, X2, X3]

for t in range(num_of_rep):
    d, w = coreset_kmeans(size, X_list, n_c = num_clusters)
    ckmeans = KMeans(n_clusters = num_clusters)
    ckmeans.fit(d,sample_weight = w)

    dist = ckmeans.transform(X_std)
    cost = np.sum((np.min(dist, axis=1)) ** 2) / num_of_data
    res[t] = cost
    print(cost)

name_str = "./res/kmeans/kmeans-coreset-"+str(num_clusters)+"c"+str(size)+"s.npy"    
np.save(name_str, res)

73.46019681608169
73.44234567211778
74.23817445185314
73.50473315261743
73.8470045225081
73.83340940298976
74.10437548157843
73.14254030775075
73.37503297719871
73.22802974817957
74.44289338836494
73.44334870173103
74.29590281199722
74.0795713427415
73.44194770990273
74.06784220645517
74.2770960732288
73.5925707763438
73.69805438857364
73.67229988940595


In [30]:
num_of_rep = 20
res = np.zeros(num_of_rep)
num_clusters = 10
size = 1200

for t in range(num_of_rep):
    d = uniform_kmeans(size, X_list, n_c = num_clusters)

    ukmeans = KMeans(n_clusters = num_clusters)
    ukmeans.fit(d)

    dist = ukmeans.transform(X_std)
    cost = np.sum((np.min(dist, axis=1)) ** 2) / num_of_data
    res[t] = cost
    print(cost)
    
name_str = "./res/kmeans/kmeans-uniform-"+str(num_clusters)+"c"+str(size)+"s.npy"    
np.save(name_str, res)

74.09765958686631
74.43134420860241
75.20928803051173
73.90252689073156
74.92758411617388
74.29166864941674
75.57129063612592
73.6838237077466
73.7346013765442
74.42756612921227
74.3671381982716
75.50098534065393
74.44264344906125
74.30753472293168
74.18747438812935
75.10918356548495
74.16285895851007
74.84423556074813
73.91553029201579
74.64825525706348


In [127]:
num_of_rep = 20
res = np.zeros(num_of_rep)
num_clusters = 10

for t in range(num_of_rep):
    dkmeans = distkmeans(X_list, np.ones(num_of_data) / num_of_data, n_c= num_clusters)

    dist = dkmeans.transform(X_std)
    cost = np.sum((np.min(dist, axis=1)) ** 2) / num_of_data
    res[t] = cost
    print(cost)

name_str = "./res/kmeans/distkmeans-"+str(num_clusters)+"c.npy"    
np.save(name_str, res)

75.46073871868204
74.68238959903377
74.90830729395657
74.73988430616967
74.53718666421031
75.84935711949967
74.56518608482786
74.64365223169615
75.20884984994078
74.70615628472683
74.87085874301633
75.24927899168051
75.13922804574419
74.87150920864161
74.65300333219258
74.36218738336815
74.68265369929877
75.35650019304215
74.73695602945946
74.50523114546864


In [9]:
num_of_rep = 20
res = np.zeros(num_of_rep)
num_clusters = 10
size = 1000

X1 = X_std[:,:30]
X2 = X_std[:,30:60]
X3 = X_std[:,60:]

X_list = [X1, X2, X3]

num_of_rep = 20
res = np.zeros(num_of_rep)
num_clusters = 10
size = 1000

for t in range(num_of_rep):
    d, w = coreset_kmeans(size, X_list, n_c = num_clusters)
    d1 = d[:,:30]
    d2 = d[:,30:60]
    d3 = d[:,60:]
    d_list = [d1,d2,d3]

    dkmeans = distkmeans(d_list, w, n_c= num_clusters)

    dist = dkmeans.transform(X_std)
    cost = np.sum((np.min(dist, axis=1)) ** 2) / num_of_data
    res[t] = cost
    print(cost)

name_str = "./res/kmeans/distkmeans-coreset-"+str(num_clusters)+"c"+str(size)+"s.npy"    
np.save(name_str, res)

76.16933092317159
76.45605367780722
77.52983651542255
76.08061189979055
76.56847816883194


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


77.25550263097848
76.42566651173429
81.19247991466676
76.284550245235
77.87532229003337
79.3088700846753
77.18928903839786


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


81.52991492224803
77.28176379745612
78.28636103886976
77.18456260234456
76.92814607141437
76.64724271583984
76.85096597898489


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


82.05321574139471


In [29]:
num_of_rep = 20
res = np.zeros(num_of_rep)
num_clusters = 10
size = 2000

for t in range(num_of_rep):
    d = uniform_kmeans(size, X_list, n_c = 10)
    d1 = d[:,:30]
    d2 = d[:,30:60]
    d3 = d[:,60:]
    d_list = [d1,d2,d3]

    dkmeans = distkmeans(d_list, np.ones(size)/size, n_c= num_clusters)

    dist = dkmeans.transform(X_std)
    cost = np.sum((np.min(dist, axis=1)) ** 2) / num_of_data
    res[t] = cost
    print(cost)

name_str = "./res/kmeans/distkmeans-uniform-"+str(num_clusters)+"c"+str(size)+"s.npy"    
np.save(name_str, res)

79.39085358827454
76.5606840728351


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


83.18637669950753


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


77.99548294217148


...

82.20262456314619


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


80.25219473048512


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


77.63127298677428
75.6717266105124
75.50828962349925
79.17990053311787
78.45062958858257
79.27173004799764


In [33]:
# change the coreset size for Coreset+K and Coreset+D
num_of_rep = 20
num_clusters = 10
size_list = [2000,3000,4000,5000,6000]

for size in size_list:
    print("coreset size %d" % size)
    res = np.zeros(num_of_rep)
    for t in range(num_of_rep):
        d, w = coreset_kmeans(size, X_list, n_c = num_clusters)
        d1 = d[:,:30]
        d2 = d[:,30:60]
        d3 = d[:,60:]
        d_list = [d1,d2,d3]

        dkmeans = distkmeans(d_list, w, n_c= num_clusters)

        dist = dkmeans.transform(X_std)
        cost = np.sum((np.min(dist, axis=1)) ** 2) / num_of_data
        res[t] = cost
        print(cost)

    name_str = "./res/kmeans/distkmeans-coreset-"+str(num_clusters)+"c"+str(size)+"s.npy"    
    np.save(name_str, res)

coreset size 2000
75.5363547527181
77.38528668315446
75.53559154011047
75.59207796213053
76.350739317434
75.77777512618921
76.69133547398981
78.13804120896341
75.8709138204326
76.17325606768101
75.66381315428715
76.1998937903758
75.62603211543279
75.80646553061983


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


78.04105303405287
77.68226659404947
75.27006407733847
75.97180795113466
76.83491150079142
76.25849345148525
coreset size 3000
77.24656687832977
74.94057169824639
75.18381189429951
75.3295937960009
75.66144378099693
76.19558364633686
75.44520705053863
75.15427070928938
75.21048867333423


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


77.0518804999432
75.11972554758319
75.39883984070383
75.51157300829038
75.10381522260461
76.06856443709773
75.26683755484503
74.89981248558833
75.04700302337615
75.61106776822957
74.93432544166811
coreset size 4000
75.80218102722738
75.19652275025298
74.84841955891132
75.88691771677671
75.73979874436826
75.29957497705124
75.62102698514562
74.95892011503688
75.12702011027417
74.90765721972211
75.60267507605069
76.05031493206421
75.38450238462289
75.8167428922834
76.18911899380858
75.94699330919715
74.86438957102115
74.93108407641333
76.22114446192295
75.3966891382549
coreset size 5000
76.11370380000415
75.62497977654564
75.21242734738549
75.79827494698822
74.72059736331603
75.3480332719696
75.21773664042418
75.03646174321243
74.47337667176993
76.20071900687743
75.71927685793742
74.62078704433387
74.91108808548769
75.5216598459384
75.51837674922942
75.00152204208328
75.88803504599571
74.94495444480945
74.85558645545872
74.7028586988418
coreset size 6000
75.14406624858219
75.4996995508876

In [34]:
num_of_rep = 20
num_clusters = 10
size_list = [2000,3000,4000,5000,6000]

for size in size_list:
    print("coreset size %d" % size)
    res = np.zeros(num_of_rep)
    for t in range(num_of_rep):
        d, w = coreset_kmeans(size, X_list, n_c = num_clusters)
        ckmeans = KMeans(n_clusters = num_clusters)
        ckmeans.fit(d,sample_weight = w)

        dist = ckmeans.transform(X_std)
        cost = np.sum((np.min(dist, axis=1)) ** 2) / num_of_data
        res[t] = cost
        print(cost)

    name_str = "./res/kmeans/kmeans-coreset-"+str(num_clusters)+"c"+str(size)+"s.npy"    
    np.save(name_str, res)

coreset size 2000
72.46134317750244
72.52116586551901
72.61014470804933
72.81573168269493
72.33371461289352
72.26337975564925
73.06211024131603
73.62089593151009
73.11230152797461
72.39632365778596
72.67820645948818
73.0021798295595
72.4452990831582
72.93485535299025
72.2830870503119
73.21546904098415
72.47913020763211
72.44097881116309
72.51648313718226
72.50294769810229
coreset size 3000
72.443642053893
72.29599203601175
72.18008283071654
72.29967256103136
72.14654954832103
72.0906063076754
72.17480883473907
72.17939671683786
72.71576122429136
72.33543159607302
72.07657268460652
72.23593545905437
72.16610404418198
71.98147442021279
72.16432199864663
72.45768627221416
72.01440183988669
72.2538161800313
72.41816291473778
72.02112919970678
coreset size 4000
72.01556694584552
72.3091542428201
72.05348581287659
72.07560139161782
72.66235345865563
71.95339056708994
72.23278218265736
72.02783723524247
72.18106017754748
72.05509406898872
71.95371265335658
72.55713881255464
72.06705877656414


In [39]:
# change the coreset size for Coreset+K and Coreset+D
num_of_rep = 20
num_clusters = 10
size_list = [1000]

for size in size_list:
    print("uniform size %d" % size)
    res = np.zeros(num_of_rep)
    for t in range(num_of_rep):
        d = uniform_kmeans(size, X_list, n_c = 10)
        d1 = d[:,:30]
        d2 = d[:,30:60]
        d3 = d[:,60:]
        d_list = [d1,d2,d3]

        dkmeans = distkmeans(d_list, np.ones(size)/size, n_c= num_clusters)

        dist = dkmeans.transform(X_std)
        cost = np.sum((np.min(dist, axis=1)) ** 2) / num_of_data
        res[t] = cost
        print(cost)

    name_str = "./res/kmeans/distkmeans-uniform-"+str(num_clusters)+"c"+str(size)+"s.npy"    
    np.save(name_str, res)

uniform size 1000
80.26167986751372
81.99329500553782
76.91167616043923
77.75682460496436
78.0466078070221
76.71822537725775
78.0477706766993


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


79.17226355741911


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


79.50080863952059


C:\Users\thoma\AppData\Local\Temp\ipykernel_25836\2868184020.py:51: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  server_kmeans.fit(center_list, sample_weight = center_weights)


79.94478846391101
77.69643234386326
81.30715834193082
79.99356675488264
78.19853178262592
79.59947040222353
78.95327589875124
77.77877250341777
78.54525663340135
76.74863838445513
80.144310014674


In [38]:
# change the coreset size for Coreset+K and Coreset+D
num_of_rep = 20
num_clusters = 10
size_list = [1000]

for size in size_list:
    print("uniform size %d" % size)
    res = np.zeros(num_of_rep)
    for t in range(num_of_rep):
        d = uniform_kmeans(size, X_list, n_c = num_clusters)

        ukmeans = KMeans(n_clusters = num_clusters)
        ukmeans.fit(d)

        dist = ukmeans.transform(X_std)
        cost = np.sum((np.min(dist, axis=1)) ** 2) / num_of_data
        res[t] = cost
        print(cost)
    
    name_str = "./res/kmeans/kmeans-uniform-"+str(num_clusters)+"c"+str(size)+"s.npy"    
    np.save(name_str, res)

uniform size 1000
74.30569766003276
73.89319646069934
75.26910626431298
76.19973633365068
73.9986084396271
75.35557061985534
74.85766155992279
75.4529064049674
75.09489693326229
74.03946914580823
75.2082713552428
76.5138539191653
73.64047519027662
75.83735016291122
75.73563004018227
73.95187054288306
74.7788408560981
73.99311710707036
74.81175722892104
75.17107539256627
